# **Assignment A3 (Part-C)**

---



In [ ]:
import pandas as pd
import numpy as np

# Load data (replace with your actual data loading method)
df_nss = pd.read_csv('NSSO68.csv')
df_nss


<ipython-input-47-58ab53b0f2b4>:5: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_nss = pd.read_csv('NSSO68.csv')


,slno,grp,Round_Centre,FSU_number,Round,Schedule_Number,Sample,Sector,state,State_Region,...,pickle_v,sauce_jam_v,Othrprocessed_v,Beveragestotal_v,foodtotal_v,foodtotal_q,state_1,Region,fruits_df_tt_v,fv_tot
0,1,40999999999999992652495293775872.0,1,41000,68,10,1,2,24,242,...,0.0,0.0,0.0,0.000000,1141.492400,30.942394,GUJ,2,12.000000,154.180000
1,2,40999999999999992652495293775872.0,1,41000,68,10,1,2,24,242,...,0.0,0.0,0.0,17.500000,1244.553500,29.286153,GUJ,2,333.000000,484.950000
2,3,40999999999999992652495293775872.0,1,41000,68,10,1,2,24,242,...,0.0,0.0,0.0,0.000000,1050.315400,31.527046,GUJ,2,35.000000,214.840000
3,4,40999999999999992652495293775872.0,1,41000,68,10,1,2,24,242,...,0.0,0.0,0.0,33.333333,1142.591667,27.834607,GUJ,2,168.333333,302.300000
4,5,40999999999999992652495293775872.0,1,41000,68,10,1,2,24,242,...,0.0,0.0,0.0,75.000000,945.249500,27.600713,GUJ,2,15.000000,148.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101657,101658,79999999999999997087170359721984.0,1,79998,68,10,1,1,1,12,...,0.0,0.0,0.0,0.000000,544.013667,28.441750,J$K,2,0.000000,25.833333
101658,101659,79999999999999997087170359721984.0,1,79998,68,10,1,1,1,12,...,0.0,0.0,0.0,8.000000,417.616600,25.490282,J$K,2,0.000000,49.000000
101659,101660,79999999999999997087170359721984.0,1,79998,68,10,1,1,1,12,...,0.0,0.0,0.0,7.142857,378.300429,25.800107,J$K,2,0.000000,32.285714
101660,101661,79999999999999997087170359721984.0,1,79998,68,10,1,1,1,12,...,0.0,0.0,0.0,14.000000,510.023600,30.220170,J$K,2,0.000000,39.200000


In [ ]:
# Data preparation (assuming similar variables as in R example)
df_ap = df_nss[df_nss['state_1'] == 'AP']

vars = ["Sector", "hhdsz", "Religion", "Social_Group", "MPCE_URP", "Sex", "Age",
        "Marital_Status", "Education", "chicken_q", "chicken_v"]

df_ap_p = df_ap[vars]

# Calculate price, handling potential division by zero
df_ap_p['price'] = np.where(df_ap_p['chicken_q'] != 0, df_ap_p['chicken_v'] / df_ap_p['chicken_q'], 0)

df_ap_p[['price']]

<ipython-input-49-be4f1ac297ea>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ap_p['price'] = np.where(df_ap_p['chicken_q'] != 0, df_ap_p['chicken_v'] / df_ap_p['chicken_q'], 0)


,price
6777,110.0
6778,110.0
6779,130.0
6780,110.0
6781,110.0
...,...
79719,130.0
79720,0.0
79721,0.0
79722,130.0


In [ ]:
from scipy.stats import describe

# Summary statistics
summary_stats = describe(df_ap_p['chicken_q'])
print(f"Summary Statistics for chicken_q:\n{summary_stats}")



Summary Statistics for chicken_q:
DescribeResult(nobs=6899, minmax=(0.0, 13.66666667), mean=0.30174615543948397, variance=0.12746968207875722, skewness=9.985607644660892, kurtosis=301.93321205798736)


DescribeResult(nobs=6899, minmax=(0.0, 13.66666667), mean=0.30174615543948397, variance=0.12746968207875722, skewness=9.985607644660892, kurtosis=301.93321205798736)

In [ ]:
import statsmodels.api as sm

# Linear regression using statsmodels
# Drop rows with NaN or inf in any column
df_ap_p_clean = df_ap_p.replace([np.inf, -np.inf], np.nan).dropna()

X = df_ap_p_clean[['hhdsz', 'Religion', 'MPCE_URP', 'Sex', 'Age', 'Marital_Status', 'Education', 'price']]
y = df_ap_p_clean['chicken_q']
X = sm.add_constant(X)  # Adding a constant (intercept) term
model = sm.OLS(y, X).fit()
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:              chicken_q   R-squared:                       0.288
Model:                            OLS   Adj. R-squared:                  0.287
Method:                 Least Squares   F-statistic:                     348.9
Date:                Mon, 01 Jul 2024   Prob (F-statistic):               0.00
Time:                        15:16:04   Log-Likelihood:                -1510.1
No. Observations:                6899   AIC:                             3038.
Df Residuals:                    6890   BIC:                             3100.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const              0.0284      0.027      1.

In [ ]:
from scipy.optimize import minimize

# Tobit regression using scipy.optimize for handling censored data
def tobit_log_likelihood(params, y, X):
    beta = params[:-2]
    sigma = np.exp(params[-2])
    gamma = params[-1]

    mu = np.dot(X, beta)
    cens_residuals = (y - mu) / sigma
    ll = np.sum(np.log(sigma) + gamma * cens_residuals - np.log(1 + np.exp(gamma * cens_residuals)))

    return -ll

# Initial guess for parameters
initial_params = np.zeros(X.shape[1] + 2)
initial_params[-2] = np.log(np.std(y))
initial_params[-1] = 1.0

# Minimize negative log-likelihood
results = minimize(tobit_log_likelihood, initial_params, args=(y, X))

# Extract estimated coefficients and standard errors
beta_est = results.x[:-2]
sigma_est = np.exp(results.x[-2])
gamma_est = results.x[-1]

# Print Tobit regression results
print("\nTobit Regression Results:")
print(f"Beta (coefficients):\n{beta_est}")
print(f"Sigma (standard deviation of residuals):\n{sigma_est}")
print(f"Gamma (Tobit parameter):\n{gamma_est}")


/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:396: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/scipy/optimize/_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:396: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
<ipython-input-45-e900b554f2ef>:6: RuntimeWarning: overflow encountered in exp
  sigma = np.exp(params[-2])
/usr/local/lib/python3.10/dist-packages/scipy/optimize/_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0



Tobit Regression Results:
Beta (coefficients):
[-0.13269726  0.06723805 -0.1390201   0.00028822 -0.18811097 -0.09154083
 -0.24325422  0.05288843 -0.00597459]
Sigma (standard deviation of residuals):
86.06524486156626
Gamma (Tobit parameter):
5.115603295006487


<ipython-input-45-e900b554f2ef>:6: RuntimeWarning: overflow encountered in exp
  sigma = np.exp(params[-2])
/usr/local/lib/python3.10/dist-packages/scipy/optimize/_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
